

A lot of code is taken from <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html"> PyTorch tutorial on Finetuning </a>

Parameters are coming from raytune.

In [1]:
## Imports
import torch
import torchvision
from torchvision.models.detection import MaskRCNN
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN
from torchvision.transforms import v2 as T
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F

import numpy as np
from pathlib import Path
import os
from facet_ml.classification import mask_rcnn
import matplotlib.pyplot as plt

In [2]:

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomVerticalFlip(0.5))
        transforms.append(T.RandomRotation(90))
        # transforms.append(T.RandomResizedCrop(size=256, scale=(0.6, 1.4)))
        transforms.append(T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1))
        # transforms.append(T.RandomCrop(size=(224, 224)))


    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [3]:
## Load Data ##

device = "cuda"
# Coco folder
coco_dir = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Coco_v5"
coco_dir_train = str( Path(coco_dir) / "train" )
coco_dir_test = str( Path(coco_dir) / "test" )

cd_train = mask_rcnn.ManualCocoColloidalDataset(coco_dir_train, 
                                             str(Path(coco_dir_train) / "_annotations.coco.json"),
                                             transforms=get_transform(True)
                                             )
cd_test = mask_rcnn.ManualCocoColloidalDataset(coco_dir_test, 
                                            str(Path(coco_dir_test) / "_annotations.coco.json"),
                                            transforms=get_transform(False)
                                            )

def collate_fn(batch):
    '''
    Collation function receives [(image_1, targets_1{masks,boxes,labels}), (image_10, targets_1{masks,boxes,labels})...]
    Need to stack image_1
    '''
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    # return tuple(zip(*batch))
    return images,targets

dataloader_train = DataLoader(
    cd_train,
    batch_size=2,
    num_workers=0,
    collate_fn=collate_fn
)
dataloader_test = DataLoader(
    cd_test,
    batch_size=2,
    num_workers=0,
    collate_fn=collate_fn
)

In [ ]:
# get the model using our helper function
num_classes=2
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.5 # .0005 starting
)
optimizer = torch.optim.Adam(
    params,
    lr=0.000165,
    betas=[0.5, 0.999],
    weight_decay=0.000618,

)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.6
)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma = .512368
)

# For testing, train it just for 10 epochs. Otherwise, 100
num_epochs = 10

metric_loggers = []
total_losses = []
from engine import train_one_epoch,evaluate

def validate(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    all_predictions = []
    
    with torch.no_grad():  # Disable gradient computation
        for images, targets in dataloader:
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Forward pass (get the model's predictions)
            outputs = model(images)
            
            # Calculate loss (same loss used in training, if applicable)
            loss_dict = model(images, targets)
            print(loss_dict)
            print(loss_dict[0].keys())
            losses = sum(loss for loss in loss_dict.values())
            val_loss += losses.item()
            
            all_predictions.append(outputs)

    avg_val_loss = val_loss / len(dataloader)
    return avg_val_loss, all_predictions

for epoch in range(num_epochs):
    # Train epochs
    metric_logger = train_one_epoch(model, optimizer, dataloader_train, device,epoch,print_freq=1)
    # metric_loggers.append( train_one_epoch(model, optimizer, dataloader_train, device,epoch,print_freq=1) )

    lr_scheduler.step()
    

In [5]:
## Target image
image_oi, targets_oi = cd_test[0]

targets_oi = [
    {
        k: v.to(device) if isinstance(v, torch.Tensor) else v
        for k, v in targets_oi.items()
    }
]
model.train()
out = model([ image_oi.to("cuda") ],targets_oi)

In [6]:
model.eval()
out = model([ image_oi.to("cuda") ])

In [ ]:
targets = targets_oi[0]
targets = out[0]
print(targets)

masks = targets["masks"]
print("masks:", masks.shape)
boxes = targets["boxes"]
labels = targets["labels"]
scores = targets.get("scores",None)


import matplotlib.patches as patches
from matplotlib.path import Path as plt_Path 
def draw_polygon_with_paths(box,ax):
    # rect = patches.Rectangle( (box[3],box[1]), box[2], box[0],face_color=None)
    vertices = [
        (box[0],box[1]), # Topleft
        (box[0],box[3]), # Bottom Left
        (box[2],box[3]), # Bottom Right
        (box[2],box[1]), # Top Right
        (box[0],box[1])  # Top Left
    ]
    codes = [
        plt_Path.MOVETO,  # Move to the first point (start)
        plt_Path.LINETO,  # Line to the second point
        plt_Path.LINETO,  # Line to the third point
        plt_Path.LINETO,  # Line to the fourth point
        plt_Path.CLOSEPOLY  # Close the path (back to the start)
    ]
    path = plt_Path(vertices, codes)
    patch = patches.PathPatch(path, facecolor='none', edgecolor='blue', linewidth=2,alpha=.1)
    ax.add_patch(patch)

def draw_mask_with_mask(mask,ax):
    from matplotlib.colors import ListedColormap, Normalize
    cmap = ListedColormap(['none', 'red'])
    ax.imshow(mask,cmap=cmap,alpha=.5)

fig, ax = plt.subplots()
ax.imshow(image_oi[0,:,:])

from IPython.display import clear_output
for ii in range(len(masks)):
    if scores is not None:
        
        if scores[ii] < .5:
            continue
    if len(masks[ii].shape) == 3:
        mask = masks[ii][0].to("cpu").detach()
    else:
        mask = masks[ii].to("cpu").detach()
    # mask = masks[ii][0].to("cpu").detach()
    box = boxes[ii].to("cpu").detach()
    label = labels[ii]
    
    draw_polygon_with_paths(box,ax)
    draw_mask_with_mask(mask,ax)
    # print(box)
    # plt.show()
    # from time import sleep
    # sleep(.1)
    # clear_output(wait=True)

plt.show()


In [ ]:
next_item = next(iter(dataloader_train))
img_load, targets_load = (next_item[0][0],next_item[1][0])
fig, ax = plt.subplots()
ax.imshow(img_load[0,:,:])

masks_load = targets_load["masks"]

for ii in range(masks_load.shape[0]):
    mask = masks_load[ii]
    draw_mask_with_mask(mask,ax)
plt.show()

In [ ]:
## Check model with completely new image
image_dir = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\Images\Additional\L1_2.5_5_10_nM\L1 2.5 nM"
image_path = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\Images\Additional\L1_2.5_5_10_nM\L1 2.5 nM\L1_2.5nM_003.jpg"

import cv2
import glob
img_list = glob.glob( str(Path(image_dir) / "*") )
im = cv2.imread(image_path)
im = cv2.imread(img_list[1])
im_torch = torch.tensor(np.moveaxis(im,-1,0)).to("cuda") / 255
out = model([im_torch])
fig, ax = plt.subplots()
ax.imshow(im[:,:,0])

masks = out[0]["masks"]
boxes = out[0]["boxes"]
scores = out[0]["scores"]

for ii,mask in enumerate(masks):
    score = scores[ii]
    if score < .5:
        continue
    draw_mask_with_mask(mask.to("cpu").detach()[0],ax)
    box = boxes[ii].to("cpu").detach()
    draw_polygon_with_paths(box,ax)

In [ ]:
len(cd_train.coco["annotations"]) + len(cd_test.coco["annotations"])

In [31]:
torch.save(model,"maskrcnn.pth")